In [1]:
from datetime import date, datetime
import math
import numpy as np, pandas as pd
import bokeh
import yaml
from bokeh.io import output_notebook, show
from bokeh.models import CustomJS, MultiSelect, Button, ColumnDataSource, DateRangeSlider
from bokeh.models import RangeSlider, CheckboxButtonGroup, MultiChoice, TextInput
from bokeh.events import ButtonClick, MenuItemClick
from bokeh.layouts import row, column
output_notebook()
ROOT.gROOT.LoadMacro("../macro/runCCDBItemList.C")
ROOT.gROOT.LoadMacro("../macro/runCCDBDownloadTarget.C")

Welcome to JupyROOT 6.26/04


Loading BokehJS ...

0

### Get the list of objects on the CCDB
First choose from where you want to get the data.
Options:
* testCCDB
* QCDB
* LocalDB

In [2]:
target = "QCDB"
path = "qc/TPC/MO/Clusters"

The /Data/UserFiles/CCDB.csv file should be generated in advance so we know what files are on the CCDB. If you already have this, no need to run the next cell which calls the runCCDBItemList.C macro.

In [ ]:
ROOT.runCCDBItemList(target,path)

### Read the file list
Here we import the file list we created into python.

In [4]:
CCDB = pd.read_csv("../../data/localDB/"+target+"list.csv",header=0, sep = r',', skipinitialspace = True)
files = list(set(CCDB.Name))
paths = list(set(CCDB.Path))
time_stamps = list(set(CCDB.TimeStamp))
types = list(set(CCDB.Type))
tasks = list(set(CCDB.Task))
sizes = list(set(CCDB.Size))
runNumbers = list(set(CCDB.RunNumber))

### Dashboard to select files to download
Click the download button after selecting the wanted task, object, file type and time range

In [5]:
files_data = CCDB.copy(deep=True)

selected_files = files_data.copy(deep=True)
shown_files = files_data.copy(deep=True)

selected_data = []
selected_types = []
selected_tasks = []
time_range = [files_data['TimeStamp'].min(),files_data['TimeStamp'].max()]
timestamp_range = 20000
runNum_range = [files_data['RunNumber'].min(),files_data['RunNumber'].max()]

In [6]:
def file_browser(doc):
    def file_size_str(table):
        size = table["Size"].sum()
        if size < 1E3:
            string = "{} Bytes".format(size)
        elif size < 1E6:
            string = "{:.2f} kB".format(size*(1E-3))
        elif size < 1E9:
            string = "{:.2f} MB".format(size*(1E-6))
        else:
            string = "{:.2f} GB".format(size*(1E-9))
        return string
    
    def download_selected(selected_files):
        ROOT.runCCDBDownloadTarget(list(selected_files["ID"]),target,"../../data/userFiles/myFile.root",)
        
    def update_button_label():
        global selected_files
        label = "Download {} files ({})".format(str(len(selected_files.index)),file_size_str(selected_files))
        button.label = label
        
    def update_file_select_list():
        global selected_files, time_range, selected_types, selected_tasks, files_data, shown_files
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        shown_files = files_data[type_condition & time_condition & task_condition]
        file_select.options = list(set(shown_files['Name']))
        
    def update_time_range():
        global shown_files
        if not shown_files.empty:
            date_range.value=[shown_files['TimeStamp'].min(),shown_files['TimeStamp'].max()]
            date_range.start=shown_files['TimeStamp'].min()
            date_range.end=shown_files['TimeStamp'].max()
    
    def update_run_range():
        global shown_files
        if not shown_files.empty:
            run_range.value=[shown_files['RunNumber'].min(),shown_files['RunNumber'].max()]
            run_range.start=shown_files['RunNumber'].min()
            run_range.end=shown_files['RunNumber'].max()
    
    def update_type_list():
        global selected_tasks, time_range, files_data
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        existing_types = files_data[time_condition & task_condition]
        type_select.options = list(set(existing_types['Type']))
        
    def update_selection():
        global selected_data, selected_files, time_range, selected_types, selected_tasks, files_data, runNum_range
        time_condition = ((files_data['TimeStamp'] >= time_range[0]) & (files_data['TimeStamp'] <= time_range[1]))
        name_condition = files_data['Name'].map(lambda x: x in selected_data)
        type_condition = files_data['Type'].map(lambda x: x in selected_types)
        task_condition = files_data['Task'].map(lambda x: x in selected_tasks)
        run_condition = ((files_data['RunNumber'] >= runNum_range[0]) & (files_data['RunNumber'] <= runNum_range[1]))
        selected_files = files_data[name_condition & time_condition & type_condition & task_condition & run_condition]
    
    def select_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, selected_tasks
        selected_data = file_select.value
        update_selection()
        update_button_label()
        
    def date_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        time_range = date_range.value
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def run_callback(attr,old,new):
        global selected_data, selected_files, files_data, runNum_range, selected_types, shown_files, selected_tasks
        runNum_range = run_range.value
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def type_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_types = type_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        update_run_range()
        
    def task_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        selected_tasks = task_select.value
        update_selection()
        update_file_select_list()
        update_time_range()
        update_button_label()
        update_type_list()
        update_run_range()
        
    def timestamp_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        global timestamp_range
        timestamp = int(timestamp_text.value)
        time_range = [timestamp-timestamp_range,timestamp+timestamp_range]
        update_selection()
        update_file_select_list()
        update_button_label()
        
    def timestamp_range_callback(attr,old,new):
        global selected_data, selected_files, files_data, time_range, selected_types, shown_files, selected_tasks
        global timestamp_range
        timestamp_range = int(timestamp_range_text.value)
        
    def button_callback(event):
        global selected_data, selected_files, files_data, time_range, selected_types
        display(selected_files)
        print("Downloading {} files".format(str(len(selected_files.index))))
        download_selected(selected_files)

    type_select = MultiChoice(title='Object type',options=types,value=[])
    type_select.on_change("value",type_callback)
    
    task_select = MultiChoice(title='QC task',options=tasks,value=[])
    task_select.on_change("value",task_callback)
        
    date_range = DateRangeSlider(title='Time range',value=[files_data['TimeStamp'].min(),files_data['TimeStamp'].max()],
                                    start=files_data['TimeStamp'].min(), end=files_data['TimeStamp'].max())
    date_range.on_change("value_throttled",date_callback)    
    
    run_range = RangeSlider(title='Run Number',value=[files_data['RunNumber'].min(),files_data['RunNumber'].max()],
                                    start=files_data['RunNumber'].min()-1, end=files_data['RunNumber'].max())
    run_range.on_change("value_throttled",run_callback)
    
    timestamp_text = TextInput(value=str(runNum_range[0]), title="Timestamp:", width=150)
    timestamp_text.on_change("value",timestamp_callback)
    
    timestamp_range_text = TextInput(value=str(20000), title="Timestamp range:", width=150)
    timestamp_range_text.on_change("value",timestamp_range_callback)
    
    file_select = MultiSelect(title="Files", value=[], options=files, height=300)
    file_select.on_change("value", select_callback)

    button = Button(label="Download all ({})".format(file_size_str(files_data)), button_type="success") 
    button.on_event(ButtonClick, button_callback)
    
    doc.add_root(row(column(task_select,type_select,date_range,run_range,row(timestamp_text,timestamp_range_text),button),file_select))
show(file_browser)

,ID,Path,Name,TimeStamp,Type,Task,Size,RunNumber
1215,1216,qc/TPC/MO/Clusters/,ClusterData,1653870918218,o2::quality_control_modules::tpc::ClustersData,Clusters,4426752,517252
1216,1217,qc/TPC/MO/Clusters/,ClusterData,1653867316050,o2::quality_control_modules::tpc::ClustersData,Clusters,3201024,517252
1217,1218,qc/TPC/MO/Clusters/,ClusterData,1653863673246,o2::quality_control_modules::tpc::ClustersData,Clusters,9361408,517247
1218,1219,qc/TPC/MO/Clusters/,ClusterData,1653860050665,o2::quality_control_modules::tpc::ClustersData,Clusters,9293824,517247
1219,1220,qc/TPC/MO/Clusters/,ClusterData,1653856450050,o2::quality_control_modules::tpc::ClustersData,Clusters,9366528,517234
1220,1221,qc/TPC/MO/Clusters/,ClusterData,1653852834694,o2::quality_control_modules::tpc::ClustersData,Clusters,9374720,517234
1221,1222,qc/TPC/MO/Clusters/,ClusterData,1653849226103,o2::quality_control_modules::tpc::ClustersData,Clusters,9371648,517234
1222,1223,qc/TPC/MO/Clusters/,ClusterData,1653845616330,o2::quality_control_modules::tpc::ClustersData,Clusters,9370624,517234
1237,1238,qc/TPC/MO/Clusters/,ClusterData,1653842008788,o2::quality_control_modules::tpc::ClustersData,Clusters,9374720,517234
1250,1251,qc/TPC/MO/Clusters/,ClusterData,1653672994283,o2::quality_control_modules::tpc::ClustersData,Clusters,9370624,517071


In [8]:
%store selected_files

Stored 'selected_files' (DataFrame)
